# Approximation of differential equations

An ***ordinary*** *differential equation (ODE) of the first order* is of the form $$\frac{dy}{dx} = f(x,y)$$ An example could be $$\frac{dy}{dx} = y$$ this doesn't need any special steps as the only function whose derivative is itself is, notably, $y = e^x (+ C)$. Doing the proper steps we'd be reordering the terms like $$\frac{dy}y = dx \longrightarrow \int \frac{dy}y = \int dx \longrightarrow \ln y = x+C \longrightarrow y = Ke^x$$ (setting $e^C = K$). So, generally speaking, when solving differential equations, the output is not a numerical value, but a function, or more precisely, a set of functions that differ by a constant (or sometimes more than one). To obtain a *unique* function we need to set a ***boundary condition***, such as $y = 1 \land x = 0$: it follows that this condition is fulfilled for $K = 1$. We also can have *initial conditions* if the pair of $x, y$ is on the integration bounds. (meaning?)

Another example is the second order ODE describing harmonic motion: $$ \frac{d^2 y}{dx^2} = -kx$$

needing 2 constraints in this case. Overall, the $n$-th order ODE needs $n$ conditions to isolate a single solution.

bluh bluh now we actually care about those that are either too analytically complex or completely lack an analytical solution. Let's work on the following generic Cauchy problem (also called Initial Value Problem): $$\begin{cases}\frac{dy}{dx} = f(x,y) \\ y(x_0) = y_0 \end{cases}$$

which is a generic first order ODE with an also generic constraint. What we care about is finding a numerical approximation for the pair $x_0, y_0$. This solution basically "samples" the real solution with a step $h$, fundamentally giving us a set of points to draw an approximated curve: $$\begin{align*}y(x_1) &= y(x_0 + h) \\ y(x_2) &= y(x_0 + 2h) \\\vdots& \\ y(x_n) &= y(x_0 + nh)\end{align*}$$ So what we do is, knowing $y_n = y(x_n)$ we want to find $y_{n+1}$. We employ a specific pair of methods, which intuitively are called **predictor-corrector (PC) methods**. From there on, $y$ will denote the exact solution, while the approximated solutions will use $w$.

## Method 1: Extrapolation through previous values (multi-step)
$$y_{n+1} = g(y_n, y_{n-1},\dots)$$
As shown above, here the n+1 step depends on _all_ of the previously obtained values.

**Pro:**
- Error order is arbitrary, how accurate the result needs to be is up to choice

**Con:**
- More complex

--- 



The analytical solution for a Cauchy problem looks like: 

$$y = \int_{x_0}^x f(x,y(x))dx$$

which numerically becomes 

$$w_{n+1} = w_n + \int_{x_n}^{x_{n+1}} f(x,y)dx$$

### Predictor step

This, very unexpectedly and without the slightest touch of sarcasm, somehow reduces into a problem of numerical integration (most of the times of the open or half-open type, because we may or may not already know one of the bounds, but it's hardly likely that we also know the second one).
Oddly enough we're changing something up: we cannot possibly know what's one index ahead, so we're using the **backward** GN formula. (shocking am I right)

$$\int_{x_n}^{x_{n+1}} f(x,y)dx = \int_0^1 \left(\sum_{k=0}^{N} r^{\overline k} \frac{\nabla^k f(x_0)}{k!}\right)hdr$$

and the reason we don't stop at a specific order is because we'll integrate for all of the $N$ previous values. The actual explicit expression of that summation is $f_n +  r\nabla f_n + \dots + \frac{r(r+1)\dots(r+n-1)}{n!} \nabla^n f_n$, compressed down to a summation cause reasons

Like usual the $f_n$ values, being constants, end up outside of the integral.



Putting things together we have the expression for $w_{n+1}$:

$$ w_{n+1} = w_n + h\int_0^1 \left(\sum_{k=0}^{N} r^{\overline k} \frac{\nabla^k f(x_0)}{k!}\right)dr $$

while the error term for the new term will be

$$\xi_{n+1} = h\int_0^1 \left(\frac{r(r+1)\dots(r+n)}{(N+1)!} \nabla^{N+1} f_n\right) dr$$

Changing the integral into an actual summation, the (n+1)th approximated term will then be
$$w_{n+1} = w_n + h \sum_{k=0}^N a_k\nabla^k f_n$$

with $f_n = f(x_n, w_n)$. $a_k$ is given by the integrals of each $r$ term, so it's $a_0 = \int_0^1 dr, a_1 = \int_0^1 rdr, a_2 = \int_0^1 r(r+1) dr\dots$


For instance, interpolating up to the first order yields the first order predictor formula:

$$w_{n+1} = w_n + h(f_n + \frac 12 \nabla f_n)= \dots (f_n + \frac 12 (f_n - f_{n-1})) = \dots (f_n + \frac 12 f_n - \frac 12 f_{n-1}) = w_n + \frac h2 (3 f_n - f_{n-1} )$$

$$\xi_n = \frac{5}{12} \nabla^2 f_n = \frac{5}{12} hh^2 f''(\xi_n) \qquad(???) \\
= \frac{5}{12} h^3 y'''(\xi_n)$$


if we attempted the interpolation up to the second order, we obtain 

$$
\begin{align*}

w_{n+1} &= w_n + h \left(f_n + \underbrace{\frac 12}_{a_1} \nabla f_n + \underbrace{\frac{5}{12}}_{a_2} \nabla^2 f_n\right) \\
&= w_n + h \left(f_n + \frac 12 (f_n - f_{n-1}) +\frac{5}{12}(f_{n} - 2f_{n-1} + f_{n-2})\right)\\
&\color{orange}= w_n + \frac{h}{12} \left(23f_n - 16f_{n-1} + 5f_{n-2}\right)


\end{align*}
$$

with error

$$\xi_n = \frac{3}{8} h^4 y^{(4)}(\xi_n)$$


Side note: we say the order of the error is equal to the power of h, minus 1. For instance, in the first order formula we have $h^3$, so the error "is of order 2", and so on.

This whole multi-step method is called **Adams-Bashforth method**. The highlighted formula is also called the "second order" (or also called "three step" because it uses three values) Adams-Bashforth method.

---



So far we've been working on $x = x_n + rh$: we move it to point $n+1$, which means $r=-1$ and the bounds of integration change. For some reason we're also changing $r$ to $s$ to avoid confusion between the two PC steps. Changing the bounds of integration implies that we cannot be lazy and reuse the previous coefficients $a_k$ obtained earlier.

### Corrector step
 
$$ w_{n+1} = w_n + \int_{-1}^0 \left(\sum_{k=0}^{N} s^{\overline k} \frac{\nabla^k f(x_0)}{k!}\right)hds \\
w_{n+1} = w_n + h \sum_{k=0}^N b_k\nabla^k f_{n+1}$$


Let's calculate the $b_k$ coefficients:

$$b_0 = 1 \\ b_1 = -\frac 12 \\ b_2 = -\frac{1}{12} \\\vdots$$

Plugging them in the previous formula at the first order

$$w_{n+1} = w_n + h \left[f_{n+1}-\frac 12(f_{n+1} - f_n)\right] = w_n + \frac h2 (f_{n+1} - f_n)$$

with error (of "order 2") $\xi = | -\frac{1}{12}h^3 y'''(\xi_{n+1})|$.

Second order interpolation yields the second order corrector formula: $$w_{n+1} = w_n + \frac{h}{12}(5f_{n+1} + 8f_n - f_{n-1})$$ with error $\xi_n = -\frac{h^4}{24} y^{(4)} (\xi_{n+1})$

***Remember that the $f_{n+1}$ term in the formulas above means $f(x_{n+1}, w_{n+1})$, but the $w_{n+1}$ in there is the one calculated from the predictor step.*** We could differentiate them by calling the corrector value $w^{c}_n$ and the predictor value, predictably, $w^{p}_n$ (great, more notation to make this harder to read). 

At every step of this iterative method, the corrector value becomes the new "predictor".

We'll iterate until $w_{n+1}^{(i)} - w_{n+1}^{(i-1)} < \varepsilon$, with $i$ being the i-th iteration.

Final note: the corrector step is, as seen so far, one step ahead, so the order is also +1 (???)

**The corrector step formula is called Adams-Moulton**.



---

Let's take

$$w_{n+1} = w_n + \frac{h}{12}(23f_n-16f_{n-1}+5f_{n-2})$$

choosing $n = 3$.

$$w_4 = w_3 + \frac{h}{12}(23f(x_3, w_3)-16f(x_2,w_2)+5f(x_1,w_1))$$

Moving on to $n = 2$, decrease those indices by 1 (and remember that $w_0 = y_0$). Clearly, $n = 1$ is not possible. To solve this problem, we need additional starting points. So-called *starting methods* can help (brief mention cause they're hard apparently)

$$w_1 = y(x_1) =  y(x_0+h) = y_0 + hy'_0 + \frac{h^2}{2}y''_0\dots$$

and $y'$ is none other than $f'$, the first equation in the example Cauchy problem we've mentioned at the start.

Same thing can be done for $y(x_2) = y(x_0 + 2h)$ and so on.

Generally:

$$y(x_0 + sh)  = y_0 + shy'_0 + \frac{(sh)^2}{2}y''_0\dots$$

---

What we're lacking in the formula above now is the value of all those derivatives: let's compute them (here we're stopping at the second nonzero value)

$$\begin{align*}
\frac{dy}{dx}&= x+y^2, y(0) = 0\\
y' &= x+y^2 \quad&\implies y'(0) = 0\\
y'' &= 1 + 2yy' \quad&\implies y''(0) = 1 \\
y''' &= 2(y')^2 + 2yy'' \quad&\implies y'''(0) = 0 \\
y^{(IV)} &= 6y'y'' + 2yy''' \quad&\implies y^{(IV)}(0) = 0 \\
y^{(V)} &= 6(y'')^2 + 6y'y''' +  2 y'y''' \quad&\implies y^{(V)}(0) = 6
\end{align*}$$

Additional explanation for the results:
In the y' line, it evaluates to zero as we already calculated that y(0) = 0, so the y^2 term will also be 0. In y'', both y' and y are 0, in y''', y and y' are 0 but y'' isn't, however the whole thing still evaluates to 0, and so on.

Explanation for the obtained derivatives:
We're applying the chain rule, as y is not just a variable but a function. So for instance, when getting the derivative of y', the y^2 term results in the derivative of the inner function (y) times the derivative of the outer function (the power of 2). And so on for the rest.

These derivative values will be plugged into the expansion we just wrote. **This is how we start our resolution using the PC method**. Set $h = 0.1$ and use the $y(x_0 + sh)$ formula above (finding values for $s = 1, 2, 3$). Skipping boring calculations, we have

$$y(0.1) = 0.0050005, y(0.2) = 0.010016, y(0.3) = 0.0451 = w_3$$

Once we have our third point, we can now evaluate the rest. Let's start with the predictor step:

$$w_{4} = w_3 + \frac{h}{12}(23f_3-16f_{2}+5f_{1})$$

Calculate $f_{1,2,3}$, considering that $"f"$ is the function we're solving for, i.e. $x+y^2$:
$$
f_3 = f(0.3, y(0.3)) = 0.3 + 0.0451^2 = 0.302034\\
f_2 = \dots = 0.02004\\
f_1 = \dots = 0.100025
$$

Plugging the result of the expression in the predictor-$w_4$: 

$$w_4^{(p)} = 0.0804376$$

This is our predictor value. Let's use it in the corrector step:

$$w_4^{(c)} = w_3 + \frac{h}{12}(5f_{4} + 8f_3 - f_{2}) = 0.0805018$$

where $f_4 = f(0.4, w_4^{(p)}) = 0.406470, f_3 = \dots$.

We can improve on the accuracy of this $w_4^{(c)}$ by recalculating $f_4$ using this newfound $w_4^{(c)}$ instead of $w_4^{(p)}$. This can be done repeatedly until the difference between each iteration becomes $< \varepsilon$.

---

The Adams-Bashforth-Moulton method can be further generalized by considering not necessarily the points $n+1$ and $n$, but also using $n-p$ as our starting point for any $p\leq n+1$. The formula (just showing the predictor) looks almost exactly the same: 

$$w_{n+1} = w_{n-p} + h \sum_{k=0}^N a_k\nabla^k f_n$$

and near same for corrector. $a_k$ is calculated the same way, with those integrals (lower bound will be $-p$).

For $p = 3$ this becomes the **Milne-Simpson** formula. The risk to this is an error that becomes exponentially larger the more solution points we try to evaluate. proof is too long and i cba

## Method 2: Extrapolation through previous value (single-step)
$$y_{n+1} = g(y_n)$$

**Pros:**
- Simpler
- Does **not** need a starting method
- **Always** stable (no Runge phenomenon, ever)

**Con:**
- The error is **not arbitrary**, the only way to reduce it is reducing $h$, you cannot reiterate in the same way as the ABM method

---

The name that the main single step PC method takes is **Runge-Kutta**.

$$w_{n+1} = w_n + h\sum_{i=0}^p \alpha_i f(x_n + \beta_i h, y_n + \gamma_i h)$$

with $\beta_0 = \gamma_0 = 0$.
These three constants are always smaller than 1.




<!-- Let's work it from the usual formula
$$y(x_n + sh)  = y_n + shy'_n + \frac{(sh)^2}{2}y''_n\dots$$ -->
This process will yield the actual values of these constants: it basically will give us the final formula (called RK2) once that's done.
Set $p = 1$
$$w_{n+1} = w_n + \alpha_0 k_0 + \alpha_1 k_1$$
the k values are determined by varying beta and gamma
$$k_0 = hf(x_n,y_n) \\ k_1 = hf(x_n + \beta_1 h, y_n + \gamma_1 k_0) \qquad \gamma_1 = \gamma f(x_n,y_n)$$

We have to determine these 4 constants: $\alpha_{0,1}, \beta, \gamma$.

$$\frac{k_1}{h} = f(x_n + \beta h, y_n + \gamma k_0) = f(\dots) + \beta h f_x + \gamma k_0 f_y + o(h^2)$$

where $f_x$ and $f_y$ are basically $f$ just differentiated along either the $x$ or $y$ axis. This is the 1st order expansion.

Expanding to second order:

$$f(\dots) + \beta h f_x + \gamma k_0 f_y + \frac 12[\beta h(\beta h f_{xx} + \gamma k_0 f_{xy}) + \gamma k_0(\beta h f_{xy} + \gamma k_0 f_{yy})] + o(h^3)$$

where $f_xx$ is the second derivative...

$$y_{n+1} = y(x_n + h) = y_n + \alpha_0 k_0 + \alpha_1 k_1 = y_n + \alpha_0 hf_n + \alpha_1 f + \alpha_1$$
$$y_{n+1}=y_n+hf+\frac 12 h^2 (f_x+ff_y)+\frac{h^3}{6}(f_{xx}+ 2ff_{xy} + f_xf_y+ff^2_y+f^2f_{yy})+\dots$$
$$w_{n+1} = w_n + \alpha_0 hf + \alpha_1 h(f+\beta h f_x +\gamma k_0 f_y + \frac 12[\beta h (\beta h f_{xx} + \gamma k_0 f_{xy}) + \gamma k_0(\beta h f_{xy} + \gamma k_0 f_{yy}))]$$

Let's compare the y and w equations: we're gonna set equalities between terms that both have the same order and same derivative (lezione14) _and skipping steps where he simplifies h and f_

order 0:

$$\alpha_0 + \alpha_1 = 1$$

order 1:

$$\alpha_1 \beta = \frac 12$$

order 2:
$$\alpha_1 \gamma = \frac 12$$

we cannot extract the third order in this case because there isn't sufficient terms in the w_n+1 equation (it's lacking one derivative(?) compared to the y equation), this problem would be "solved" by solving instead the $p=2$ case.

this is basically a system of equations: there's 3 equations and 4 parameters, Rouché-Capelli's theorem teaches us that one of these is then **arbitrary**.

$$
\begin{cases}
a_1 = c \\
a_0 = 1-c \\
\beta = \frac 1{2 c} \\
\gamma = \frac 1{2 c} \\
\end{cases}
$$
and for c=1/2

$$w_{n+1} = w_n + \frac 12 (k_0+k_1)$$



go fuck yourself this whole ordeal is BAD and we're never using it anyway it's called heun's method, also called RK2

constants k are calculated in order because each k requires the previous constant

### Runge-Kutta 4 (RK4)

finally going to the useful thing we're going as far as p=3 and skipping the proof cause it's boring as shit and thankfully prof said so
$$w_{n+1} = w_n + \frac 16 (k_0+2k_1+2k_2+k_3)$$

$$k_0 = hf(x_n,y_n)\\ k_1 = hf\left(x_n + \frac h2, y_n + \frac {k_0}2 \right) \\ k_2 = hf\left(x_n + \frac h2, y_n + \frac {k_1}2 \right) \\ k_3 = hf\left(x_n + h, y_n + k_2 \right)

insert how to get stability of RK4 (lezione 15 forse)